In [1]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(torch.cuda.current_device()))

2.3.1+cu121
True
0
NVIDIA RTX A5500


In [2]:
import os
import yaml

def show_yaml_file(yaml_file):
    if os.path.exists(yaml_file):
        try:
            with open(yaml_file, 'r') as file:
                data = yaml.safe_load(file)
            print("YAML file content:\n")
            print(yaml.dump(data, default_flow_style=False))
        except Exception as e:
            print(f"Error reading YAML file: {e}")
    else:
        print("File doesn't exist.")

# Specify the path to your yaml file
yaml_file_path = r'C:\Users\priya\OneDrive\Desktop\yolo_vd\custom_dataset\data.yaml'
show_yaml_file(yaml_file_path)


File doesn't exist.


In [3]:
import os
import torch
from torchvision import transforms
from ultralytics import YOLO

# Load the model
model = YOLO('yolov8s.pt')
device = 'cuda:0'

# Define transformations
transform = transforms.Compose([
    transforms.Resize((640, 640)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Define checkpoint save function
def save_checkpoint(model, epoch, path='C:\\Users\\priya\\OneDrive\\Desktop\\yolo_vd\\custom_dataset\\checkpoints'):
    os.makedirs(path, exist_ok=True)
    torch.save(model.state_dict(), os.path.join(path, f'checkpoint_epoch_{epoch}.pth'))

# Early stopping class
class EarlyStopping:
    def __init__(self, patience=10, min_delta=0.001):
        self.patience = patience
        self.min_delta = min_delta
        self.best_loss = None
        self.counter = 0

    def check(self, current_loss):
        if self.best_loss is None:
            self.best_loss = current_loss
        elif current_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                return True
        else:
            self.best_loss = current_loss
            self.counter = 0
        return False

# Custom training function with checkpoint saving and early stopping
def train_with_early_stopping(model, data, epochs, batch_size, lr, device, checkpoint_interval=10, patience=10):
    model.to(device)
    
    early_stopping = EarlyStopping(patience=patience)

    # Configure and start training
    model.train(
        data=data,
        epochs=epochs,
        batch=batch_size,
        imgsz=640,
        device=device,
        optimizer='Adam',  # Explicitly set the optimizer
        lr0=lr,
        lrf=0.01,
        fliplr=0.0,
        amp=False
    )

    # Training loop for checkpoints and early stopping
    for epoch in range(epochs):
        if (epoch + 1) % checkpoint_interval == 0:
            save_checkpoint(model, epoch + 1)

        # Perform validation after each epoch
        results = model.val()
        current_loss = results['metrics/loss']

        # Check for early stopping
        if early_stopping.check(current_loss):
            print(f"Early stopping at epoch {epoch + 1}")
            break

# Define paths and parameters
data_path = r'C:\Users\priya\OneDrive\Desktop\yolo_vd\custom_dataset\data.yaml'
epochs = 50
batch_size = 2
lr = 0.0001

# Run training with early stopping and checkpoint saving every 10 epochs
train_with_early_stopping(
    model,
    data=data_path,
    epochs=epochs,
    batch_size=batch_size,
    lr=lr,
    device=device,
    checkpoint_interval=10  # Set to save checkpoints every 10 epochs
)


New https://pypi.org/project/ultralytics/8.2.82 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.60  Python-3.11.9 torch-2.3.1+cu121 CUDA:0 (NVIDIA RTX A5500, 24564MiB)
engine\trainer: task=detect, mode=train, model=yolov8s.pt, data=C:\Users\priya\OneDrive\Desktop\yolo_vd\custom_dataset\data.yaml, epochs=50, time=None, patience=100, batch=2, imgsz=640, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=None, name=train5, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=False, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retin

RuntimeError: Dataset 'C://Users/priya/OneDrive/Desktop/yolo_vd/custom_dataset/data.yaml' error  'C:\Users\priya\OneDrive\Desktop\yolo_vd\custom_dataset\data.yaml' does not exist

In [ ]:
import os
import cv2
from ultralytics import YOLO

# Paths provided by you
model_weights = r"C:\Users\priya\OneDrive\Desktop\yolo_vd\runs\detect\train33\weights\best.pt"
source_path = r"C:\Users\debas\OneDrive\Desktop\vd_be\file.mp4"
output_path = r"C:\Users\priya\OneDrive\Desktop\yolo_vd\OUTPUT_VID.mp4"

# Function to run inference on a video and save the output
def run_inference(model_weights, source, output):
    # Load the model
    model = YOLO(model_weights)
    
    # Open the input video
    cap = cv2.VideoCapture(source)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return
    
    # Get video properties
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    
    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output, fourcc, fps, (width, height))
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Run YOLOv8 inference on the frame
        results = model(frame)
        
        violence_detected = False
        for result in results:
            for box in result.boxes:
                confidence = box.conf.item()  # Get confidence score
                
                if box.cls == 1 and confidence > 0.60:  # Only consider violence with confidence > 65%
                    violence_detected = True
                    
                    # Draw bounding box
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
                    
                    # Overlay the label "Violence detected" with accuracy
                    label = f"Violence detected ({confidence * 100:.2f}%)"
                    cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
        
        if not violence_detected:
            # Overlay the label "Non-violence detected" if no violence is detected
            cv2.putText(frame, "Non-violence detected", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        
        # Write the frame to the output video
        out.write(frame)
    
    # Release everything when job is finished
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    
    print(f"Inference completed. Output saved to {output}")

if __name__ == "__main__":
    run_inference(model_weights, source_path, output_path)


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\priya\\OneDrive\\Desktop\\yolo_vd\\runs\\detect\\train33\\weights\\best.pt'